http://zeag.farbox.com/post/fen-ci-ren-wu-step-by-step

In [1]:
# -*- coding: utf-8 -*-
"""
@author: zeag
"""

import numpy as np

class Node:
    def __init__(self,s, from_node=None, dis=0):
        self.s = s#节点字符串
        self.from_node = from_node#字符串前驱
        self.dis = dis#从始点到该节点的最短距离


class Segdemo:
    def __init__(self):
        self.word_dic = {'#开始#':2514605,'他':2826, '说':2556, '的':54478, '的确':27,'确':27,'确实':53, '实':31, '实在':35, '在':12023, '在理':1,'理':28}
        self.trans = {'#开始#@他':1069,'他@说':355,'说@的':43,'的@确实':1 }

    def conventToWordNet(self,s):#建立词网
        word_net = []
        word_net.append([Node('#开始#')])
        start = 0
        while(start < len(s)):
            tmp = []
            for end in range(start,len(s)):
                tmp_str = s[start:end+1]
                if tmp_str in self.word_dic:
                    tmp.append(Node(tmp_str))
                else:
                    break
            word_net.append(tmp)
            start += 1
        word_net.append([Node("#结束#")])
        return word_net

    def calcDistance(self, from_node, to):#计算节点和节点的距离
        frequency = self.word_dic[from_node]
        nTwoWordsFreq = self.trans.get(from_node +'@'+to,0)
        value = -np.log(0.1 * frequency / 2.5146057e7 + 0.9 * (0.9999899602323339 * nTwoWordsFreq / frequency + 1.00e-5))
        #这个公式是用hanlp中的公式，这个公式加了一些平滑因子，总体思路就是转移次数越多，距离越小。
        if value < 0:
            value = -value
        return value

    def viterbi(self, word_net):
        word_list = []
        for i,item in enumerate(word_net):
            if i == len(word_net)-1:
                break
            for from_node in item:
                if from_node.s == "#开始#":
                    to_nodes = word_net[i+1]
                else:
                    to_nodes = word_net[i + len(from_node.s)]
                for to_node in to_nodes:
                    dis = from_node.dis + self.calcDistance(from_node.s, to_node.s)
                    if to_node.from_node == None or to_node.dis > dis:
                        to_node.from_node = from_node
                        to_node.dis = dis
        word = word_net[-1][0]
        while(word.from_node.s != "#开始#"):
            word_list.append(word.from_node.s)
            word = word.from_node
        return word_list[::-1]



if __name__ == '__main__':
    s = "他说的确实在理"
    demo = Segdemo()
    word_net = demo.conventToWordNet(s)
    word_list = demo.viterbi(word_net)
    print(word_list)

['他', '说', '的', '确实', '在理']
